In [4]:
from spotipy import Spotify
import spotipy
import spotipy.util as util
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame as df
import pprint
import json
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
if __name__ == "__main__":
    user = #'spotify username of interest here'
    client_id = #'spotify app client_id here'
    secret = #'spotify app secret here'
    scope = 'user-read-recently-played'
    token = util.prompt_for_user_token(user, scope, client_id, secret, 'http://localhost:8888/callback/')
    sp = spotipy.Spotify(auth=token)
    
    if token:
        recent = sp.current_user_recently_played(limit=50)

In [613]:
song_counter = 0
title = []
artist = []
artist_id = []
playtime = []
song_id = []

while song_counter < 50:
    title.append(recent['items'][song_counter]['track']['name'])
    artist.append(recent['items'][song_counter]['track']['artists'][0]['name'])
    artist_id.append(recent['items'][song_counter]['track']['artists'][0]['id'])
    playtime.append(recent['items'][song_counter]['played_at'])
    song_id.append(recent['items'][song_counter]['track']['id'])
    song_counter += 1

title_df = pd.DataFrame(title)
artist_df = pd.DataFrame(artist)
artist_id_df = pd.DataFrame(artist_id)
playtime_df = pd.DataFrame(playtime)
song_id_df = pd.DataFrame(song_id)

transfer = [title_df, artist_df, artist_id_df, playtime_df, song_id_df]
playback_raw = pd.concat(transfer, axis=1, sort=False)
playback_raw.columns = ['title','artist','artist_id','playtime','song_id']
playback_raw['playtime'] = pd.to_datetime(playback_raw['playtime'])
playback_raw['playtime'] = playback_raw['playtime'].dt.round('1s')
playback_new = playback_raw.drop_duplicates(subset='playtime', keep='first')

In [614]:
query_songs = list(playback_new.song_id)

In [615]:
audio_df = pd.DataFrame()

if token:
    for i in query_songs:
        audio = sp.audio_features(i)
        audio_df = audio_df.append(audio)
        
audio_df.rename(columns={'id' : 'song_id'}, inplace=True)

In [617]:
playback_ready = playback_new.merge(audio_df, on='song_id')

In [618]:
merge_df = pd.DataFrame()
artist_input = list(playback_ready['artist_id'])

In [619]:
art_query = sp.artists(artist_input)
aq = 0

artist_id = []
genres = []
popularity = []

while aq < len(artist_input):
    genres.append(art_query['artists'][aq]['genres'])
    artist_id.append(art_query['artists'][aq]['id'])
    popularity.append(art_query['artists'][aq]['popularity'])
    aq += 1
    
artist_id_df = pd.DataFrame(artist_id)
genre_df = pd.Series(genres)
popularity_df = pd.DataFrame(popularity)
transfer2 = [artist_id_df, genre_df, popularity_df]
artist_raw = pd.concat(transfer2, axis=1, sort=False)
artist_raw.columns = ('artist_id', 'genres', 'popularity')

In [ ]:
merge_df = playback_ready.merge(artist_raw)
final_df = merge_df.drop_duplicates('playtime', keep='first')

In [ ]:
final_df.head(5)